<a href="https://colab.research.google.com/github/eshikaalam/multilabelEmotionRoBERTa/blob/main/multilabel5withaccuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [3]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, Subset
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from torch.optim import AdamW
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import time
import datetime
from torch.utils.data import Dataset
from torch.utils.data import random_split
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('Using GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('Using CPU')

Using GPU: Tesla T4


In [5]:
df = pd.read_csv("/content/drive/MyDrive/dataset/MONOVAB.csv")

print(df.columns)  # See the actual column names

Index(['Unnamed: 0', 'comment', 'anger', 'contempt', 'disgust', 'enjoyment',
       'fear', 'sadness', 'surprise'],
      dtype='object')


In [6]:
df = pd.read_csv("/content/drive/MyDrive/dataset/MONOVAB.csv")

# Drop any rows with missing values if necessary
df.dropna(inplace=True)

# Text input
texts = df["comment"].tolist()

# Multi-label emotion targets
labels = df[["anger", "contempt", "disgust", "enjoyment", "fear", "sadness", "surprise"]].values.astype(float)


In [7]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=64):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float)

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )

        return (
            encoding["input_ids"].squeeze(0),
            encoding["attention_mask"].squeeze(0),
            label
        )

dataset = EmotionDataset(texts, labels, tokenizer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [8]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


In [9]:
class BengaliEmotionClassifier(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super(BengaliEmotionClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained("xlm-roberta-base")
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.bert.config.hidden_size, 7)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]  # CLS token
        pooled_output = self.dropout(pooled_output)
        return self.classifier(pooled_output)


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BengaliEmotionClassifier().to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.BCEWithLogitsLoss()

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round(elapsed))))

batch_size = 8  # adjust if you still face memory issues
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

epochs = 4


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [12]:
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
import time
import datetime
import torch

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round(elapsed))))

epochs = 4

for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    t0 = time.time()

    model.train()
    total_loss = 0

    for batch in train_loader:
        b_input_ids, b_input_mask, b_labels = [x.to(device) for x in batch]

        model.zero_grad()
        outputs = model(b_input_ids, attention_mask=b_input_mask)
        loss = loss_fn(outputs, b_labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        del b_input_ids, b_input_mask, b_labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_loader)
    print("Average Train Loss:", avg_train_loss)
    print("Epoch Training Time:", format_time(time.time() - t0))

    # =================== Validation ===================
    model.eval()
    val_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            b_input_ids, b_input_mask, b_labels = [x.to(device) for x in batch]

            outputs = model(b_input_ids, attention_mask=b_input_mask)  # ✅ FIXED: removed .logits
            loss = loss_fn(outputs, b_labels)
            val_loss += loss.item()

            preds = torch.sigmoid(outputs).detach().cpu().numpy()
            labels = b_labels.detach().cpu().numpy()

            all_preds.append(preds)
            all_labels.append(labels)

            del b_input_ids, b_input_mask, b_labels, outputs, loss
            torch.cuda.empty_cache()

    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    binary_preds = (all_preds >= 0.5).astype(int)

    f1_macro = f1_score(all_labels, binary_preds, average='macro')
    f1_micro = f1_score(all_labels, binary_preds, average='micro')
    accuracy = accuracy_score(all_labels, binary_preds)

    print(f"Validation Loss: {val_loss / len(val_loader):.4f}")
    print(f"F1 Score (Macro): {f1_macro:.4f}")
    print(f"F1 Score (Micro): {f1_micro:.4f}")
    print(f"Accuracy: {accuracy:.4f}")



Epoch 1/4
Average Train Loss: 0.36671036946517166
Epoch Training Time: 0:02:58
Validation Loss: 0.3191
F1 Score (Macro): 0.3139
F1 Score (Micro): 0.5542
Accuracy: 0.3721

Epoch 2/4
Average Train Loss: 0.3070153267771909
Epoch Training Time: 0:02:56
Validation Loss: 0.2832
F1 Score (Macro): 0.3390
F1 Score (Micro): 0.5693
Accuracy: 0.4254

Epoch 3/4
Average Train Loss: 0.2769404262712158
Epoch Training Time: 0:02:56
Validation Loss: 0.2736
F1 Score (Macro): 0.3735
F1 Score (Micro): 0.6222
Accuracy: 0.4973

Epoch 4/4
Average Train Loss: 0.2513959427616347
Epoch Training Time: 0:02:56
Validation Loss: 0.2716
F1 Score (Macro): 0.4654
F1 Score (Micro): 0.6408
Accuracy: 0.5046
